<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=62d24d94123ed7536bd5942bc43515544ca2065200ddc14afe98d927fb8da510
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-28 14:44:15
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -23.17 K (-0.16%)
Current PnL: -22.63 L (-14.83%)
CY Booked + Current PnL: -8.74 L (-5.72%)
-------------------
Total profit:  1.65 L
Total loss:  -24.28 L
-------------------
Total Booked + Current PnL: 18.24 L (14.52%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.7%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.04 L (61.49%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.41%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.17,-7.95,8.65,0.02,13899.0,-13874.0,160685.0,144.98,68.0,M-SC,10.50,234.0,-1.00,1.12,39.77,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-2.60,-16.35,19.66,0.09,16573.0,-16479.0,84298.0,95.73,38.0,M-SC,3.50,253.0,-0.99,0.59,10.75,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,-1.86,7.33,11.52,19.69,23708.0,14058.0,205800.0,-3.15,49.0,X-MC,3.31,78.0,0.59,1.44,14.63,XY25,NTT,AC
43,ITC,452.00,-0.28,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,44.0,X-LC,1.21,5.0,-0.13,1.38,3.98,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.14,5.52,13.91,20.19,25587.0,9625.0,183945.0,-17.20,46.0,X-MC,7.22,68.0,0.38,1.28,21.47,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RAJOOENG,143.10,3.51,-24.82,85.70,39.61,66040.0,-25440.0,77060.0,-49.12,40.0,H-SC,20.48,135.0,-0.39,0.54,8.61,AR,ATH,MISC
82,VBL,671.64,3.22,-2.57,39.24,35.67,120757.0,-8103.0,307739.0,-14.21,63.0,X-LC,5.71,4.0,-0.07,2.15,10.93,X40N,ATH,FMCG
14,CAMPUS,393.00,1.94,-6.24,42.16,33.28,66318.0,-10475.0,157300.0,-24.45,53.0,M-SC,2.93,222.0,-0.16,1.10,21.92,XY24,NTT,FOOTWEAR
51,MASFIN,398.61,1.51,-5.39,29.00,22.05,26883.0,-5280.0,92700.0,-18.25,50.0,H-SC,7.00,164.0,-0.20,0.65,34.75,XR,ATH,FINANCE
63,SATIN,274.00,1.46,-14.71,78.03,51.83,183867.0,-40648.0,235636.0,-23.78,58.0,H-SC,1.61,146.0,-0.22,1.65,14.79,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIS,528.00,-3.90,-23.91,59.37,21.26,50157.0,-26550.0,84482.0,1993.26,47.0,H-SC,3.86,166.0,-0.53,0.59,14.14,OX40N,NTT,MISC
78,TTKPRESTIG,770.00,-2.60,-16.35,19.66,0.09,16573.0,-16479.0,84298.0,95.73,38.0,M-SC,3.50,253.0,-0.99,0.59,10.75,OX40N,NTT,DURABLES
6,ASIANTILES,137.00,-2.41,-14.15,111.62,81.67,89895.0,-13273.0,80537.0,7238.89,47.0,L-SC,19.47,271.0,-0.15,0.56,55.96,XR,NTT,CERAMICS
72,TANLA,1943.92,-2.31,-37.22,249.94,119.69,424856.0,-100778.0,169983.0,-36.76,30.0,H-SC,10.31,156.0,-0.24,1.19,34.29,AR,ATH,IT
44,JCHAC,2282.00,-1.89,-28.94,40.71,-0.01,36973.0,-36984.0,90821.0,18486.96,13.0,M-SC,1.19,233.0,-1.00,0.63,2.84,OX40N,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.87,0.70,105.28,106.72,130758.0,864.0,124200.0,-50.06,41.0,H-SC,2.81,139.0,0.01,0.87,24.24,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.39,-0.78,36.69,35.63,76297.0,-1624.0,207950.0,-14.97,44.0,H-MC,3.77,119.0,-0.02,1.45,15.77,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.58,-5.10,119.08,107.91,166854.0,-7525.0,140119.0,-22.58,35.0,M-SC,10.68,216.0,-0.05,0.98,0.19,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.17,-7.95,8.65,0.02,13899.0,-13874.0,160685.0,144.98,68.0,M-SC,10.50,234.0,-1.00,1.12,39.77,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-2.60,-16.35,19.66,0.09,16573.0,-16479.0,84298.0,95.73,38.0,M-SC,3.50,253.0,-0.99,0.59,10.75,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.29,-21.22,44.04,13.47,93557.0,-57231.0,212436.0,-67.53,38.0,H-SC,1.60,158.0,-0.61,1.48,8.02,XY24,NTT,PAINTS
67,SIS,528.00,-3.90,-23.91,59.37,21.26,50157.0,-26550.0,84482.0,1993.26,47.0,H-SC,3.86,166.0,-0.53,0.59,14.14,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.18,2.54,68.00,72.27,118150.0,4302.0,173750.0,-11.98,60.0,M-LC,3.68,99.0,0.04,1.21,8.87,XR,NTT,IT
38,INDIAMART,4810.62,0.87,0.70,105.28,106.72,130758.0,864.0,124200.0,-50.06,41.0,H-SC,2.81,139.0,0.01,0.87,24.24,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.18,2.54,68.00,72.27,118150.0,4302.0,173750.0,-11.98,60.0,M-LC,3.68,99.0,0.04,1.21,8.87,XR,NTT,IT
38,INDIAMART,4810.62,0.87,0.70,105.28,106.72,130758.0,864.0,124200.0,-50.06,41.0,H-SC,2.81,139.0,0.01,0.87,24.24,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.39,-0.78,36.69,35.63,76297.0,-1624.0,207950.0,-14.97,44.0,H-MC,3.77,119.0,-0.02,1.45,15.77,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.58,-5.10,119.08,107.91,166854.0,-7525.0,140119.0,-22.58,35.0,M-SC,10.68,216.0,-0.05,0.98,0.19,OX40N,ATH,CABLES
1,ABB,7934.00,-1.25,-5.05,53.31,45.57,132422.0,-13219.0,248400.0,-38.95,54.0,H-MC,5.77,120.0,-0.10,1.74,5.70,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.02,-35.50,109.62,35.20,90976.0,-45678.0,82992.0,8.88,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,600.00,-0.32,-15.49,68.23,42.18,159389.0,-42804.0,233606.0,-24.30,27.0,X-LC,1.95,3.0,-0.27,1.63,1.19,XY24,NTT,AUTO
46,JSWINFRA,342.00,0.04,-9.08,26.74,15.23,48707.0,-18198.0,182149.0,-22.32,28.0,X-MC,6.92,66.0,-0.37,1.27,20.24,X40N,NTT,REALTY
9,BAJAJHFL,181.50,-0.24,-14.04,72.69,48.45,125291.0,-28142.0,172364.0,-20.30,31.0,X-MC,7.54,64.0,-0.22,1.20,0.13,X40N,ATH,FINANCE
55,PGHH,17907.65,0.12,-4.43,39.96,33.76,76693.0,-8895.0,191925.0,-32.64,34.0,X-MC,3.87,57.0,-0.12,1.34,0.54,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.28,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,44.0,X-LC,1.21,5.0,-0.13,1.38,3.98,X40,NTT,FMCG
3,ACC,3906.0,-1.12,-22.29,111.23,64.15,205687.0,-53031.0,184920.0,-54.57,49.0,X-SC,1.58,82.0,-0.26,1.29,3.73,XY24,BTT,CEMENT
15,CAMS,4762.0,-0.63,1.42,23.05,24.79,58874.0,3564.0,255420.0,-3.25,48.0,X-SC,1.86,86.0,0.06,1.78,24.88,X40N,NTT,MISC
66,SIEMENS,4671.5,-0.65,-11.59,41.97,25.51,69051.0,-21570.0,164525.0,0.93,63.0,H-LC,1.87,49.0,-0.31,1.15,21.24,AR,ATH,ELECTRICAL
76,TMPV,600.0,-0.32,-15.49,68.23,42.18,159389.0,-42804.0,233606.0,-24.30,27.0,X-LC,1.95,3.0,-0.27,1.63,1.19,XY24,NTT,AUTO


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.02,-35.50,109.62,35.20,90976.0,-45678.0,82992.0,8.88,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-1.49,-6.31,34.56,26.07,53015.0,-10340.0,153400.0,-31.05,34.0,X-MC,9.58,55.0,-0.20,1.07,0.00,X40,ATH,APPARELS
9,BAJAJHFL,181.50,-0.24,-14.04,72.69,48.45,125291.0,-28142.0,172364.0,-20.30,31.0,X-MC,7.54,64.0,-0.22,1.20,0.13,X40N,ATH,FINANCE
55,PGHH,17907.65,0.12,-4.43,39.96,33.76,76693.0,-8895.0,191925.0,-32.64,34.0,X-MC,3.87,57.0,-0.12,1.34,0.54,X40,ATH,FMCG
76,TMPV,600.00,-0.32,-15.49,68.23,42.18,159389.0,-42804.0,233606.0,-24.30,27.0,X-LC,1.95,3.0,-0.27,1.63,1.19,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,0.04,-27.98,98.34,42.84,46039.0,-18190.0,46816.0,-52.62,41.0,X-SC,37.58,83.0,-0.40,0.33,1.87,XY24,NTT,MISC
60,RELAXO,1176.00,-0.31,-47.04,187.60,52.31,144214.0,-68287.0,76873.0,-43.66,37.0,X-SC,4.53,91.0,-0.47,0.54,2.90,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.02,-35.50,109.62,35.20,90976.0,-45678.0,82992.0,8.88,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.56,-6.49,30.02,21.58,39341.0,-9090.0,131049.0,-52.88,35.0,X-MC,6.42,56.0,-0.23,0.92,16.02,X40N,ATH,IT
52,MEDANTA,1486.00,-0.62,3.72,19.63,24.08,25847.0,4723.0,131673.0,-4.69,47.0,X-SC,5.89,89.0,0.18,0.92,24.07,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.21,-14.60,39.67,19.28,117209.0,-50495.0,295461.0,-25.57,59.0,X-LC,7.11,1.0,-0.43,2.06,9.23,X40,ATH,IT
41,INFY,2275.00,-0.21,6.41,45.54,54.87,151621.0,20052.0,332940.0,-17.09,61.0,X-LC,2.47,2.0,0.13,2.33,15.22,X40,BTT,IT
76,TMPV,600.00,-0.32,-15.49,68.23,42.18,159389.0,-42804.0,233606.0,-24.30,27.0,X-LC,1.95,3.0,-0.27,1.63,1.19,XY24,NTT,AUTO
82,VBL,671.64,3.22,-2.57,39.24,35.67,120757.0,-8103.0,307739.0,-14.21,63.0,X-LC,5.71,4.0,-0.07,2.15,10.93,X40N,ATH,FMCG
43,ITC,452.00,-0.28,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,44.0,X-LC,1.21,5.0,-0.13,1.38,3.98,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.07,-36.65,109.78,32.89,53692.0,-28298.0,48909.0,-703.98,65.0,L-MC,5.76,259.0,-0.53,0.34,34.69,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.41,-14.15,111.62,81.67,89895.0,-13273.0,80537.0,7238.89,47.0,L-SC,19.47,271.0,-0.15,0.56,55.96,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.07,-11.80,58.21,39.54,51937.0,-11935.0,89224.0,-31.03,67.0,M-SC,6.67,220.0,-0.23,0.62,25.81,AR,ATH,AUTO
51,MASFIN,398.61,1.51,-5.39,29.00,22.05,26883.0,-5280.0,92700.0,-18.25,50.0,H-SC,7.00,164.0,-0.20,0.65,34.75,XR,ATH,FINANCE
70,SURYODAY,216.00,0.79,-19.26,52.38,23.03,75734.0,-34486.0,144585.0,57.89,49.0,H-SC,9.59,167.0,-0.46,1.01,43.46,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.17,-7.95,8.65,0.02,13899.0,-13874.0,160685.0,144.98,68.0,M-SC,10.50,234.0,-1.00,1.12,39.77,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.14,0.49,20.36,20.95,51511.0,1232.0,253000.0,-0.49,75.0,X-LC,15.17,31.0,0.02,1.77,36.30,X40,ATH,PAINTS
50,LTIM,7201.88,1.04,11.30,18.30,31.66,50135.0,27809.0,273960.0,7.16,66.0,H-LC,13.58,43.0,0.55,1.91,52.01,X200,ATH,IT
79,UNITDSPR,1644.00,0.68,6.93,12.94,20.77,32397.0,16219.0,250363.0,-2.32,63.0,X-MC,3.79,62.0,0.50,1.75,14.32,X40N,NTT,BREWERIES
40,INDUSINDBK,1800.00,0.07,-36.65,109.78,32.89,53692.0,-28298.0,48909.0,-703.98,65.0,L-MC,5.76,259.0,-0.53,0.34,34.69,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.95
1,25,44.87
2,50,75.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.46
MC    29.18
LC    25.37
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.83
X40      21.25
X40N     11.99
XR        9.52
XY25      9.09
AR        9.08
OX40N     7.60
X200      1.91
SR        0.98
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.90
X-MC    22.70
X-LC    20.13
M-SC    11.92
X-SC     8.24
H-MC     4.75
H-LC     3.06
L-SC     1.40
M-MC     1.39
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.86,-4.28,38.00
IT,13.81,-14.43,73.51
FINANCE,9.50,-13.96,62.12
MISC,7.36,-23.35,73.89
ELECTRICAL,5.99,-9.54,49.66
PAINTS,5.98,-7.83,23.85
INSURANCE,4.40,-0.21,35.52
PHARMA,3.90,0.06,31.93
AUTO,3.37,-18.89,69.86


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3052624.0,21
XR,1299335.0,13
AR,1273507.0,10
X40,996517.0,14
X40N,731779.0,9
OX40N,703913.0,10
XY25,339856.0,6
SR,280687.0,2
MH,75245.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3559931.0,25
M-SC,1351684.0,15
X-MC,1231528.0,16
X-LC,881553.0,11
X-SC,740286.0,8
H-MC,398939.0,3
L-SC,261362.0,3
H-LC,119186.0,2
M-LC,118150.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1210102.0      6
           AR         875094.0      5
           XR         795093.0      7
M-SC       XY24       779800.0      6
X-MC       X40        439562.0      7
X-LC       X40        389874.0      5
X-MC       XY24       366404.0      3
H-SC       OX40N      323710.0      4
X-SC       X40N       295632.0      3
M-SC       OX40N      292486.0      5
H-SC       SR         280687.0      2
X-SC       XY24       277573.0      3
X-MC       X40N       245736.0      4
X-LC       XY24       228525.0      2
H-MC       AR         208719.0      2
X-LC       X40N       190411.0      2
H-MC       XY24       190220.0      1
X-MC       XY25       179826.0      2
L-SC       XR         173645.0      2
X-SC       X40        167081.0      2
M-SC       XR         158755.0      2
           AR         120643.0      2
M-LC       XR         118150.0      1
L-SC       OX40N       87717.0      1
H-SC       MH          75245.0      1
X-LC       XY25        72743.0      2
H-LC       AR          69051.0      1
L-MC       XR          53692.0      1
H-LC       X200        50135.0      1
M-MC       XY25        46692.0      1
L-LC       XY25        40595.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
